## CartPole의 상태

CartPole의 상태는 observation 변수에 저장되어 있다. observation 변수는 네 개의 변수로 구성된 리스트 형태로서 각 변수는 다음과 같다.

* 수레의 위치: -2.4 ~ 2.4

* 수레의 속도: -inf ~ inf

* 봉의 각도: -41.8도 ~ 41.8

* 봉의 각속도: -inf ~ inf

CartPole의 상태는 미로와 달리 변수가 여러 개이고 모두 연속적인 값을 갖는다. CartPole의 형태는 미로보다 복잡한 형태를 갖지만 미로 태스크와 마찬가지로 Q함수를 표형식으로 나타내려면 연속값을 이산값으로 변환해야한다.

* -2.4 ~ -1.6 = 0

* -1.6 ~ -0.8 = 1

* -0.8 ~ 0.0 = 2

* 0.0 ~ 0.8 = 3

* 0.8 ~ 1.6 = 4

* 1.6 ~ 2.4 = 5

그러나 범위를 벗어날 가능성도 있으므로 -inf 부터 -1.6 사이를 0으로 두고, 1.6부터 inf 사이를 5로 정의한다

* -inf ~ -1.6 = 0

* -1.6 ~ -0.8 = 1

* -0.8 ~ 0.0 = 2

* 0.0 ~ 0.8 = 3

* 0.8 ~ 1.6 = 4

* 1.6 ~ inf = 5

다른 변수도 동일한 방법으로 6개의 구간을 갖는 이산변수로 변환한다. 따라서 변수의 가짓수는 네 개이므로 모두 6^4 = 1296개의 상태로 수레의 상태를 알 수 있다.

CartPole에서 취할 수 있는 행동은 수레를 오른쪽과 왼쪽으로 미는 두 가지 방법뿐이다. 그러므로 각 행동은 수레에 왼쪽 혹은 오른쪽으로 가속도를 부여하게 되므로 각 상태변수를 6개의 구간으로 갖는 이산변수로 변환하면 CartPole의 Q함수는 1296행 2열로 된 데이터프레임으로 나타낼 수 있다. 미로 태스크와 마찬가지로 Q함수 표의 값은 각 상태에서 각 행동을 취했을 때 받게 되는 할인보상의 합이다.

## 상태의 이산변수 변환 구현

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gym

%matplotlib inline

In [2]:
'''
상수 정의
'''

# 태스크 네임
ENV = 'CartPole-v0'

# 각 상태를 이산변수로 변환할 구간 수
NUM_DIZITIZED = 6

In [4]:
'''
CartPole 실행
'''

# 태스크 실행 환경 생성
env = gym.make(ENV)

# 태스크 환경 초기화, 변수 상태 저장
observation = env.reset()

In [9]:
print(observation)
print(observation.shape)
print(type(observation))

[ 0.04960587 -0.04738796 -0.03448114  0.03166491]
(4,)
<class 'numpy.ndarray'>


In [12]:
'''
obervation 변수 값을 이산변수로 변환하는 함수를 정의하기 전에 bins를 정의한다. bins 함수는 이산값을 정하기 위한 구간을 계산하는 역할을 한다.

np.linspace는 각 구간 경계값으로 이루어진 수열을 생성하는 명령어다.

처음 부분과 마지막 부분의 값은 -inf, inf 값이 들어가야 하므로 첫 번째 요소와 마지막 요소를 뺀 부분의 리스트를 생성한다.
'''

def bins(clip_min, clip_max, num):
    
    # 관측된 상태(연속값)을 이산 값으로 변환
    term = np.linspace(clip_min, clip_max, num + 1)[1: -1]
    
    return term

print(np.linspace(-2.4, 2.4, 6 + 1))
print(np.linspace(-2.4, 2.4, 6 + 1)[1: -1])

[-2.4 -1.6 -0.8  0.   0.8  1.6  2.4]
[-1.6 -0.8  0.   0.8  1.6]


In [15]:
'''
bins 함수를 통해 구한 값에 따라 연속변수를 이산변수로 변환하는 함수 digitize_state를 구현한다.
'''

# 관측된 상태(observation)를 이산값으로 치환
def digitize_state(observation):
    
    cart_pos, cart_v, pole_angle, pole_v = observation
    
    digitized = [
        np.digitize(
            cart_pos, 
            bins = bins(-2.4, 2.4, NUM_DIZITIZED)
        ),
        
        np.digitize(
            cart_v, 
            bins = bins(-3.0, 3.0, NUM_DIZITIZED)
        ),
        
        # pole_angle 변수 구간이 -0.5부터 0.5인 이유는 pole_angle 값의 단위가 라디안이기 때문
        np.digitize(
            pole_angle, 
            bins = bins(-0.5, 0.5, NUM_DIZITIZED) 
        ),
        
        np.digitize(
            pole_v, 
            bins = bins(-2.0, 2.0, NUM_DIZITIZED)
        )
    ]
    
    out = sum([x * (NUM_DIZITIZED**i) for i, x in enumerate(digitized)])
    
    return out

In [16]:
out = digitize_state(observation)

In [17]:
print(out)
print(out.shape)
print(type(out))

735
()
<class 'numpy.int64'>
